In [2]:
from sklearn.decomposition  import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston, load_iris, load_breast_cancer, make_blobs
import numpy as np
from random import randrange
from sklearn.metrics import accuracy_score
import cvxpy as cp

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = load_iris()
X, y = data.data, data.target

# PCA

In [3]:
class PrincipalComponentAnalysis():
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
    
    def fit(self, X):
        U, S, Vh = np.linalg.svd(X, full_matrices=False) 
        self.components = Vh.T[:, :self.n_components]
    
    def transform(self, X):
        return X @ self.components

In [4]:
X_centered = X - np.mean(X, axis=0)

pca = PCA(n_components=3)
pca.fit(X_centered)
X_pca_trans = pca.transform(X_centered)

model = PrincipalComponentAnalysis(n_components=3)
model.fit(X_centered)
X_model_trans = model.transform(X_centered)

sign_correct_X_model_trans = np.concatenate([np.expand_dims(X_model_trans[:,0],axis=1),-X_model_trans[:,1:]],axis=1)

error = rel_error(X_pca_trans, sign_correct_X_model_trans)
print(error)
assert  error < 1e-11

1.5392603984884818e-13


# Binary Linear SVM with CVXPY

## Hard margin 

In [5]:
X2, y2 = make_blobs(n_samples=300, centers=2, n_features=12, random_state=47)
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)
y2[y2 == 0] = -1

$$\min_{\mathbf{w},b}\frac{1}{2}||\mathbf{w}||^2$$ <br>
$$\text{s.t } y_i(\mathbf{w}^{\top}\mathbf{x}_i + b) \ge 1, \ i=1...N$$

In [6]:
class LinearSVMHardMargin():
    def __init__(self):
        self.w = None
        self.b = 0
    
    def fit(self, X, y):
        N, D = X.shape
        self.w = cp.Variable((D,))
        self.b = cp.Variable((N,))
        constraint = [cp.multiply(y, ( X @ cp.transpose(self.w) + self.b)) >= 1]
        obj = cp.Minimize(0.5 * cp.sum_squares(self.w))
        prob = cp.Problem(obj, constraint)
        prob.solve()
        
    def predict(self, X):
        """Return the predicted label 1 or -1"""
        temp = X @ np.transpose(self.w.value) + self.b.value
        y = np.where( temp < 0, -1, 1)
        return y

In [7]:
model = LinearSVMHardMargin()
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
print(accuracy)
assert accuracy == 1

1.0


## Soft Margin

In [8]:
data3 = load_breast_cancer()
X3, y3 = data3.data, data3.target
scaler = StandardScaler()
X3 = scaler.fit_transform(X3)
y3[y3 == 0] = -1

$$L(\mathbf{w},b) = \frac{1}{N} \sum_{i=1}^N \max(0, y_i(\mathbf{w}^{\top}\mathbf{x}_i + b)) + \lambda||\mathbf{w}||^2_2$$

In [9]:
class LinearSVMSoftMargin():
    def __init__(self, alpha=0):
        self.w = None
        self.b = 0
        self.alpha = alpha
    
    def fit(self, X, y):
        N, D = X.shape
        self.w = cp.Variable(D)
        self.b = cp.Variable(N)
        hinge = cp.sum(cp.pos(1 - cp.multiply(y, ( X @ cp.transpose(self.w) + self.b))))
        reg = self.alpha * cp.pnorm(self.w, 2) ** 2
        obj = cp.Minimize((1 / N) * hinge + reg)
        prob = cp.Problem(obj)
        prob.solve()
        print("status", prob.status)
        
    def predict(self, X):
        """Return the predicted label 1 or -1"""
        temp = X @ np.transpose(self.w.value) + self.b.value
        y = np.where( temp < 0, -1, 1)
        return y

In [10]:
model = LinearSVMSoftMargin(alpha=1e-3)
model.fit(X3, y3)
pred = model.predict(X3)
accuracy = accuracy_score(y3, pred)
print(accuracy)
assert accuracy >= 0.98

status optimal
1.0


# Multiclass Linear SVM

## Loss

$$L(\mathbf{W}) = \sum_{i=1}^N \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + 1) + \lambda||\mathbf{w}||^2_2$$ <br>
$$\text{where } s_j = (f(\mathbf{x}_i;\mathbf{W}))_j = (\mathbf{W}\mathbf{x}_i)_j \text{ is the score for the j-th class}$$

In [11]:
W = np.random.randn(X.shape[1], 3) * 0.0001

In [111]:
def svm_loss_naive(W, X, y, alpha):
    """
    Multiclass SVM loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    
    N = len(X)
    C = len(W[0])
    s = X @ W

    for i in range(N):
        loss_i = 0
        for j in range(C):
            if j == y[i]:
                continue
            else:
                loss_i += max(0, (s[i, j] - s[i, y[i]] + 1))
                if s[i, j] - s[i, y[i]] + 1 > 0:
                    dW[:, y[i]] -= X[i, :] 
                    dW[:, j] += X[i, :] 
        loss += loss_i
    loss += alpha * np.linalg.norm(W) ** 2
    dW += 2 * alpha * W

    return loss, dW

In [112]:
# NO REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 0.0)

f = lambda W: svm_loss_naive(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -75.300000 analytic: -75.300000, relative error: 9.229459e-11
numerical: 125.600000 analytic: 125.600000, relative error: 1.614516e-11
numerical: 143.000000 analytic: 143.000000, relative error: 3.945470e-11
numerical: 12.500000 analytic: 12.500000, relative error: 2.392040e-10
numerical: 125.600000 analytic: 125.600000, relative error: 1.614516e-11
numerical: 43.100000 analytic: 43.100000, relative error: 2.352284e-11
numerical: 344.400000 analytic: 344.400000, relative error: 1.263701e-11
numerical: -124.000000 analytic: -124.000000, relative error: 1.738366e-12
numerical: -13.900000 analytic: -13.900000, relative error: 6.669146e-11
numerical: 125.600000 analytic: 125.600000, relative error: 1.614516e-11
numerical: 12.500000 analytic: 12.500000, relative error: 2.392040e-10
numerical: -13.900000 analytic: -13.900000, relative error: 6.669146e-11


In [113]:
#REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 2)

f = lambda W: svm_loss_naive(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -55.599441 analytic: -55.599441, relative error: 1.158107e-11
numerical: 43.099863 analytic: 43.099863, relative error: 4.813819e-12
numerical: 43.099863 analytic: 43.099863, relative error: 4.813819e-12
numerical: -269.099641 analytic: -269.099641, relative error: 1.534289e-11
numerical: -123.999492 analytic: -123.999492, relative error: 6.644347e-12
numerical: -111.699769 analytic: -111.699769, relative error: 1.577981e-11
numerical: -269.099641 analytic: -269.099641, relative error: 1.534289e-11
numerical: -13.899200 analytic: -13.899200, relative error: 1.115228e-10
numerical: -13.899200 analytic: -13.899200, relative error: 1.115228e-10
numerical: -123.999492 analytic: -123.999492, relative error: 6.644347e-12
numerical: -269.099641 analytic: -269.099641, relative error: 1.534289e-11
numerical: -13.899200 analytic: -13.899200, relative error: 1.115228e-10


In [100]:
def svm_loss_vectorized(W, X, y, alpha):
    """
    Multiclass SVM loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    N, D = X.shape
    D, C = W.shape
    s = np.zeros((N, C))
    diff = np.zeros((N, C))
    s_y = np.zeros((N, C))
    
    # loss computation 
    s = X @ W
    s_y = s[np.arange(N), y] 
    s_y_full = np.full((C, N), s_y)
    diff = np.maximum(0, s - s_y_full.T + 1)
    diff[np.arange(N), y] = 0
    loss = np.mean(np.sum(diff, axis=1)) + alpha * np.linalg.norm(W) ** 2
    
    # gradient computation
    diff[diff > 0] = 1
    temp = np.sum(diff, axis=1)
    diff[np.arange(N), y] = -temp.T
    dW = X.T @ diff
    dW = (dW / N) + 2 * alpha * W

    return loss, dW

In [101]:
# NO REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 0.0)

f = lambda W: svm_loss_vectorized(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 0.287333 analytic: 0.287333, relative error: 7.293566e-12
numerical: -0.092667 analytic: -0.092667, relative error: 2.355969e-11
numerical: -0.370667 analytic: -0.370667, relative error: 8.586207e-12
numerical: 2.296000 analytic: 2.296000, relative error: 2.062713e-12
numerical: 0.083333 analytic: 0.083333, relative error: 1.299523e-10
numerical: 0.287333 analytic: 0.287333, relative error: 7.293566e-12
numerical: 0.287333 analytic: 0.287333, relative error: 7.293566e-12
numerical: 0.837333 analytic: 0.837333, relative error: 2.003241e-12
numerical: -0.744667 analytic: -0.744667, relative error: 1.263768e-11
numerical: -0.502000 analytic: -0.502000, relative error: 1.327170e-11
numerical: -0.744667 analytic: -0.744667, relative error: 1.263768e-11
numerical: 2.296000 analytic: 2.296000, relative error: 2.062713e-12


In [102]:
# REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 2)

f = lambda W: svm_loss_vectorized(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -0.502717 analytic: -0.502717, relative error: 1.359264e-11
numerical: -0.370108 analytic: -0.370108, relative error: 8.594677e-12
numerical: -0.091867 analytic: -0.091867, relative error: 9.159680e-12
numerical: 0.836797 analytic: 0.836797, relative error: 4.491058e-14
numerical: 2.295715 analytic: 2.295715, relative error: 3.359131e-12
numerical: -0.826159 analytic: -0.826159, relative error: 2.162724e-11
numerical: 0.287196 analytic: 0.287196, relative error: 2.260099e-12
numerical: -0.502717 analytic: -0.502717, relative error: 1.359264e-11
numerical: 0.836797 analytic: 0.836797, relative error: 4.491058e-14
numerical: -1.793641 analytic: -1.793641, relative error: 8.356813e-13
numerical: 0.953895 analytic: 0.953895, relative error: 1.121822e-11
numerical: 0.953895 analytic: 0.953895, relative error: 1.121822e-11


## Gradient descent

In [127]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            X = np.insert(X, 0, 1, axis=1)
            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            indices = np.random.choice(N, batch_size, replace=False)
            X_batch = X[indices]
            y_batch = y[indices]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            self.W -= learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class LinearSVM(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return svm_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        if self.fit_intercept:
            X = np.insert(X, 0, 1, axis=1)
        y_pred_score = X @ self.W
        y_pred = np.argmax(y_pred_score, axis=1)
        return y_pred

In [128]:
model = LinearSVM(fit_intercept=True)
model.train(X, y, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X)
model_accuracy = accuracy_score(y, pred)
print(model_accuracy)
assert model_accuracy > 0.97

iteration 0 / 75000: loss 2.000112
iteration 10000 / 75000: loss 0.142495
iteration 20000 / 75000: loss 0.083408
iteration 30000 / 75000: loss 0.099203
iteration 40000 / 75000: loss 0.089971
iteration 50000 / 75000: loss 0.074385
iteration 60000 / 75000: loss 0.100232
iteration 70000 / 75000: loss 0.131352
0.9733333333333334
